In [1]:
# Set the seed value for the notebook so the results are reproducible
from numpy.random import seed
seed(1)

In [2]:
# Dependencies
import numpy as np
import pandas as pd
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
import tensorflow
tensorflow.keras.__version__

'2.2.4-tf'

In [4]:
url = 'https://raw.githubusercontent.com/lucyhohler/Final-Project---Data-Analytics/master/Resources/ufo-sightings.csv'
df = pd.read_csv(url, sep=",", na_filter=False)
df.head()

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude,month,season,time of day,part of day
0,1/10/1947 20:00,conroe,tx,us,other,4,3 to4 sec.,The date was early 1947 before Arnold&#39s &qu...,4/16/2005,30.3116667,-95.4558333,1,Winter,8:00 PM,EVENING
1,1/10/1959 18:30,south san francisco,ca,us,disk,120,2 minutes,Its was approximately 6:30pm&#44 I was sitting...,4/27/2004,37.6547222,-122.4066667,1,Winter,6:30 PM,EVENING
2,1/10/1964 20:00,el cerrito (over old berkeley dump),ca,us,fireball,120,2 minutes,bal of fire in the sky no reported air crashes...,5/15/2006,37.9158333,-122.3105556,1,Winter,8:00 PM,EVENING
3,1/10/1980 0:00,exeter,nh,us,disk,7800,10min to 2hrs,saw what appeared as very bright star&#44 some...,2/18/2001,42.9813889,-70.9483333,1,Winter,12:00 AM,NIGHT
4,1/10/1981 20:30,glendora,ca,us,unknown,60,1 minute,On a very foggy night in 1981 I was illuminate...,2/18/2001,34.1361111,-117.8644444,1,Winter,8:30 PM,EVENING


In [5]:
data = df.drop(columns=["duration (seconds)", "duration (hours/min)", "comments", "latitude", "longitude ", "time of day"])
df1 = df.drop(columns=["datetime", "city", "country", "duration (seconds)", "duration (hours/min)", "comments", "date posted", "latitude", "longitude ", "month", "time of day"])
data.head()

,datetime,city,state,country,shape,date posted,month,season,part of day
0,1/10/1947 20:00,conroe,tx,us,other,4/16/2005,1,Winter,EVENING
1,1/10/1959 18:30,south san francisco,ca,us,disk,4/27/2004,1,Winter,EVENING
2,1/10/1964 20:00,el cerrito (over old berkeley dump),ca,us,fireball,5/15/2006,1,Winter,EVENING
3,1/10/1980 0:00,exeter,nh,us,disk,2/18/2001,1,Winter,NIGHT
4,1/10/1981 20:30,glendora,ca,us,unknown,2/18/2001,1,Winter,EVENING


In [6]:
df1['outcome'] = '1'

import math

for index, row in df1.iterrows():
    outcome = '1' 
    #print(row['shape'])
    if not row['shape'] or row['shape'] == 'other' or row['shape'] == 'unknown':
        outcome = '0'
    row['outcome'] = outcome

df1.head()

,state,shape,season,part of day,outcome
0,tx,other,Winter,EVENING,0
1,ca,disk,Winter,EVENING,1
2,ca,fireball,Winter,EVENING,1
3,nh,disk,Winter,NIGHT,1
4,ca,unknown,Winter,EVENING,0


In [7]:
from sklearn.preprocessing import LabelEncoder

data = data.apply(LabelEncoder().fit_transform)
data.head()

,datetime,city,state,country,shape,date posted,month,season,part of day
0,372,2140,45,1,21,162,1,4,2
1,373,9525,5,1,13,173,1,4,2
2,374,2942,5,1,15,190,1,4,2
3,375,3193,31,1,13,105,1,4,4
4,376,3801,5,1,29,105,1,4,2


In [8]:
# Assign X (data) and y (target)
X = data
y = df1["outcome"]
print(X.shape, y.shape)

(65033, 9) (65033,)


In [9]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1, stratify=y)

In [11]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [12]:
# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [13]:
# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [15]:
# Create model and add layers
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=9))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.


In [16]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [17]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               1000      
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 202       
Total params: 11,302
Trainable params: 11,302
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2
)

Instructions for updating:
Use tf.cast instead.
Epoch 1/60
 - 3s - loss: 0.3659 - acc: 0.8536
Epoch 2/60
 - 3s - loss: 0.3158 - acc: 0.8636
Epoch 3/60
 - 2s - loss: 0.2940 - acc: 0.8611
Epoch 4/60
 - 2s - loss: 0.2414 - acc: 0.8699
Epoch 5/60
 - 2s - loss: 0.1514 - acc: 0.9619
Epoch 6/60
 - 3s - loss: 0.0813 - acc: 0.9962
Epoch 7/60
 - 2s - loss: 0.0359 - acc: 0.9998
Epoch 8/60
 - 2s - loss: 0.0153 - acc: 1.0000
Epoch 9/60
 - 2s - loss: 0.0059 - acc: 1.0000
Epoch 10/60
 - 3s - loss: 0.0024 - acc: 1.0000
Epoch 11/60
 - 3s - loss: 0.0103 - acc: 0.9977
Epoch 12/60
 - 3s - loss: 9.1201e-04 - acc: 1.0000
Epoch 13/60
 - 3s - loss: 7.6150e-04 - acc: 1.0000
Epoch 14/60
 - 3s - loss: 6.1556e-04 - acc: 1.0000
Epoch 15/60
 - 3s - loss: 4.1756e-04 - acc: 1.0000
Epoch 16/60
 - 3s - loss: 2.4302e-04 - acc: 1.0000
Epoch 17/60
 - 3s - loss: 1.2399e-04 - acc: 1.0000
Epoch 18/60
 - 3s - loss: 0.0525 - acc: 0.9959
Epoch 19/60
 - 3s - loss: 1.1127 - acc: 0.9310
Epoch 20/60
 - 3s - loss: 1.1127 - acc: 0.93

In [19]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

 - 0s - loss: 2.2186 - acc: 0.8624
Normal Neural Network - Loss: 2.2186157462106975, Accuracy: 0.8623531460762024


In [20]:
encoded_predictions = model.predict_classes(X_test_scaled[:20])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [21]:
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:20])}")

Predicted classes: ['1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1'
 '1' '1']
Actual Labels: ['0', '1', '0', '1', '1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1']
